### 1. Import thư viện cần thiết

In [ ]:
from pyspark.sql import SparkSession

### 2. Khởi tạo Spark 

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IE212_LAB02_Bai1") \
    .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/11/25 21:56:26 WARN Utils: Your hostname, ELUCIDATOR resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/25 21:56:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 21:56:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 3. Load dữ liệu

In [3]:
movies_path = "data/movies.txt"
ratings1_path = "data/ratings_1.txt"
ratings2_path = "data/ratings_2.txt"

In [4]:
movies_rdd = sc.textFile(movies_path)

In [5]:
ratings_rdd = sc.textFile(f"{ratings1_path},{ratings2_path}")

### 4. Giải quyết bài toán

##### 4.1. Các hàm hỗ trợ

In [6]:
def parse_rating_line(line):
    """
    Input: "UserID,MovieID,Rating,Timestamp"
    Output: (MovieID, (Rating, 1))
    """
    parts = line.split(",")
    movie_id = parts[1].strip()
    rating = float(parts[2])
    # Trả về (MovieID, (Rating, 1)) để dùng cho việc tính trung bình
    return (movie_id, (rating, 1))

In [7]:
def parse_movie_line(line):
    """
    Input: "MovieID,Title,Genres"
    Output: (MovieID, Title)
    """
    parts = line.split(",")
    movie_id = parts[0].strip()
    title = parts[1].strip()
    return (movie_id, title)

##### 4.2. Xử lý tính toán

In [8]:
# Bước A: Parse và Map dữ liệu ratings
# Output: (MovieID, (Rating, 1))
rating_pairs = ratings_rdd.map(parse_rating_line)

In [9]:
# Bước B: Reduce để tính tổng điểm và tổng lượt đánh giá
# Cộng dồn Rating (x[0]+y[0]) và cộng dồn số lượt đếm (x[1]+y[1])
rating_sums = rating_pairs.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [10]:
# Bước C: Tính trung bình (Average)
# Output: (MovieID, (AvgRating, TotalCount))
movie_stats = rating_sums.mapValues(lambda v: (v[0] / v[1], v[1]))

In [11]:
# Bước D: Parse dữ liệu movies để lấy tên phim
# Output: (MovieID, Title)
movies_info = movies_rdd.map(parse_movie_line)

In [12]:
# Bước E: Join dữ liệu thống kê với tên phim
# movies_info: (ID, Title)
# movie_stats: (ID, (Avg, Count))
# joined_data: (ID, (Title, (Avg, Count)))
joined_data = movies_info.join(movie_stats)

### 5. Hiển thị kết quả

In [ ]:
# Yêu cầu 1: In danh sách các phim và tính điểm trung bình

formatted_rdd = joined_data.map(lambda x: (x[1][0], x[1][1][0], x[1][1][1])) # (Title, Avg, Count)
results = formatted_rdd.collect()

for title, avg_rating, total_ratings in results:
    print(f"{title} AverageRating: {avg_rating:.2f} (TotalRatings: {total_ratings})")

The Terminator (1984) AverageRating: 4.06 (TotalRatings: 18)
Fight Club (1999) AverageRating: 3.50 (TotalRatings: 7)
Gladiator (2000) AverageRating: 3.61 (TotalRatings: 18)
Psycho (1960) AverageRating: 4.00 (TotalRatings: 2)
The Godfather: Part II (1974) AverageRating: 4.00 (TotalRatings: 17)
Sunset Boulevard (1950) AverageRating: 4.36 (TotalRatings: 7)
E.T. the Extra-Terrestrial (1982) AverageRating: 3.67 (TotalRatings: 18)
The Lord of the Rings: The Fellowship of the Ring (2001) AverageRating: 3.89 (TotalRatings: 18)
The Lord of the Rings: The Return of the King (2003) AverageRating: 3.82 (TotalRatings: 11)
The Social Network (2010) AverageRating: 3.86 (TotalRatings: 7)
Mad Max: Fury Road (2015) AverageRating: 3.47 (TotalRatings: 18)
The Silence of the Lambs (1991) AverageRating: 3.14 (TotalRatings: 7)
Lawrence of Arabia (1962) AverageRating: 3.44 (TotalRatings: 18)
No Country for Old Men (2007) AverageRating: 3.89 (TotalRatings: 18)


In [14]:
# Yêu cầu 2: Tìm phim có điểm cao nhất (với điều kiện >= 5 lượt đánh giá)
# Lọc phim có TotalRatings >= 5
candidates_rdd = formatted_rdd.filter(lambda x: x[2] >= 5)

if not candidates_rdd.isEmpty():
    # Tìm phim có AvgRating lớn nhất
    best_movie = candidates_rdd.max(key=lambda x: x[1])
    
    best_title = best_movie[0]
    best_avg = best_movie[1]
    
    # Format output theo yêu cầu đề bài
    print(f"{best_title} is the highest rated movie with an average rating of {best_avg:.2f} among movies with at least 5 ratings.")
else:
    print("\nNot Found")

Sunset Boulevard (1950) is the highest rated movie with an average rating of 4.36 among movies with at least 5 ratings.
